In [19]:
import os
import math
import json
import re
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn import tree
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
import joblib
import numpy as np
import pandas as pd
import random
from keybert import KeyBERT
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import Counter
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
import tensorflow as tf
import datetime
from tensorflow.keras.models import load_model

In [12]:
df = pd.read_csv('Datasets/dsc_fc_summed_spectra_2022_v01/dsc_fc_summed_spectra_2022_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)
df_ = pd.read_csv('Datasets/dsc_fc_summed_spectra_2022_v01/dsc_fc_summed_spectra_2022_v01.csv', delimiter = ',', parse_dates=[0], infer_datetime_format=True, na_values='0', header = None)

In [3]:
# df[53].count()

df_interp = df[[47, 48, 49, 50, 51, 52, 53]]
# px.line(df_interp[[47, 48, 49, 50, 51, 52, 53]], x=df_interp.index, y=df_interp.columns)

In [10]:
fig = go.Figure()

# Plot each column separately, dropping nan values
for col in df_interp.columns:
    temp_df = df_interp[[col]].dropna()
    fig.add_trace(go.Scatter(x=temp_df.index, y=temp_df[col], mode='lines', name=col))

# fig.show()
pyo.plot(fig, filename='Plots/lastcolumns.html', auto_open=False)

'Plots/lastcolumns.html'

In [29]:
with open('Datasets/KIndex/2015_DGD.txt', 'r') as file:
    lines = file.readlines()[12:]

df = pd.DataFrame(columns=["DateTime", "MiddleLatitudeA", "MiddleLatitudeK", "HighLatitudeA", "HighLatitudeK", "EstimatedPlanetaryA", "EstimatedPlanetaryK"])
for line in lines:
    ln = line.replace("-", " -").split()
    dt = " ".join(ln[:3])
    for i in range(8):
        df.loc[len(df)] = {"DateTime" : (dt + " " +str(i*3)), "MiddleLatitudeA": ln[3], "MiddleLatitudeK": ln[4 + i], "HighLatitudeA": ln[12], "HighLatitudeK": ln[13 + i], "EstimatedPlanetaryA": ln[21], "EstimatedPlanetaryK": ln[22 + i]}


# ln = lines[0].split()
# dt = " ".join(ln[:3])
# for i in range(8):
#     df.loc[len(df)] = {"DateTime" : (dt + " " +str(i*3)), "MiddleLatitudeA": ln[3], "MiddleLatitudeK": ln[4 + i], "HighLatitudeA": ln[12], "HighLatitudeK": ln[13 + i], "EstimatedPlanetaryA": ln[21], "EstimatedPlanetaryK": ln[22 + i]}
# ln = lines[1].split()
# dt = " ".join(ln[:3])
# for i in range(8):
#     df.loc[len(df)] = {"DateTime" : (dt + " " +str(i*3)), "MiddleLatitudeA": ln[3], "MiddleLatitudeK": ln[4 + i], "HighLatitudeA": ln[12], "HighLatitudeK": ln[13 + i], "EstimatedPlanetaryA": ln[21], "EstimatedPlanetaryK": ln[22 + i]}

In [ ]:
def handle_anomalies(value):
    if value == "-1":
        return np.nan
    elif value == "":
        return np.nan
    elif value == "2-1":  # Handle the "2-1" anomaly
        return 2
    else:
        return int(value)

def process_line_final(line):
    # Extract year, month, and day
    year, month, day = line[:4], line[5:7], line[8:10]

    # Extract the A-indices and K-indices for each location
    fredericksburg_a, *fredericksburg_k = split_indices(line[11:43])
    college_a, *college_k = split_indices(line[44:76])
    planetary_a, *planetary_k = split_indices(line[77:109])

    # Handle anomalies and convert to appropriate data type
    fredericksburg_a = handle_anomalies(fredericksburg_a)
    fredericksburg_k = [handle_anomalies(k) for k in fredericksburg_k]

    college_a = handle_anomalies(college_a)
    college_k = [handle_anomalies(k) for k in college_k]

    planetary_a = handle_anomalies(planetary_a)
    planetary_k = [handle_anomalies(k) for k in planetary_k]

    # Return the extracted data
    return {
        "Date": f"{year}-{month}-{day}",
        "Middle Latitude Frederiksburg A-index": fredericksburg_a,
        "Middle Latitude Frederiksburg K-indices": fredericksburg_k,
        "High Latitude College A-index": college_a,
        "High Latitude College  K-indices": college_k,
        "Estimated planetary A-index": planetary_a,
        "Estimated planetary K-indices": planetary_k
    }

# Read the file and process each line
with open("/mnt/data/2015_DGD.txt", "r") as file:
    # Skip the header lines
    for _ in range(12):
        next(file)

    # Process the remaining lines
    data = [process_line_final(line) for line in file if line.strip()]

# Convert the processed data into a pandas DataFrame
df = pd.DataFrame(data)

df.head()


# Read the file and process each line again
with open("/mnt/data/2015_DGD.txt", "r") as file:
    # Skip the header lines
    for _ in range(12):
        next(file)

    # Process the remaining lines
    data = [process_line_final(line) for line in file if line.strip()]

# Convert the processed data into a pandas DataFrame
df = pd.DataFrame(data)

df.head()